In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
interesting=pd.read_excel('interesting_sources.xlsx')
interesting.head()

,index,source_id,ra,dec,mean_raw_peak_flux,std_raw_peak_flux,Mod,length,sig,grad,fit_error,y_int,avg_sig,avg_grad,avg_fit_error,avg_y_int,avg_std,avg_median,avg_Mod
0,2490,9626,203.152823,-62.778506,0.765738,0.303694,39.660271,94,3.983624,0.000438,0.000110,-1074.363164,1.504227,0.000356,0.000237,-874.196992,0.252276,0.697101,36.189312
1,2676,10057,141.356216,-62.528659,0.344761,0.101120,29.330620,36,8.899923,0.000300,0.000034,-737.705074,5.960626,0.000328,0.000055,-805.570112,0.110607,0.339858,32.545129
2,3289,11643,168.081167,-61.342199,0.848598,0.337037,39.716871,75,6.032429,0.000696,0.000115,-1710.497678,2.617023,0.000529,0.000202,-1300.078987,0.241509,0.759459,31.800188
3,3294,11656,201.677521,-61.331384,0.827691,0.411734,49.744936,38,-0.079716,-0.000022,0.000275,54.666834,0.451813,0.000200,0.000443,-491.436920,0.314633,0.792390,39.706788
4,3482,12131,243.560256,-61.021059,2.573485,1.978635,76.885424,70,1.129493,0.000947,0.000838,-2323.165422,-0.670967,-0.001408,0.002098,3461.891655,1.738782,2.528826,68.758477


In [14]:
def query(ra, dec, radius=5):
    url = f"http://simbad.u-strasbg.fr/simbad/sim-coo?Coord={ra}d{dec}d&CooFrame=FK5&CooEpoch=2000&CooEqui=2000&CooDefinedFrames=none&Radius={radius}&Radius.unit=arcmin&submit=submit+query"
    return url

In [22]:
def get_response(ra, dec, radius=5):

    url = query(ra, dec, radius)
    
    response = requests.get(url)

    tables = pd.read_html(response.text)
    
    idx = np.argmax(['Identifier' in t.columns for t in tables])

    table = tables[idx]
    
    try:
        
        identifier = table.loc[0, 'Identifier']
        
    except KeyError:
        identifier = 'No object found'

    return identifier, url, table

In [23]:
ra, dec = interesting.loc[1, ['ra', 'dec']]
print(query(ra, dec))
i, u, t = get_response(ra, dec)
print(i)
t

http://simbad.u-strasbg.fr/simbad/sim-coo?Coord=141.356216d-62.528659d&CooFrame=FK5&CooEpoch=2000&CooEqui=2000&CooDefinedFrames=none&Radius=5&Radius.unit=arcmin&submit=submit+query
TYC 8944-2490-1


,N,Identifier,dist(asec),Otype,ICRS (J2000)RA,ICRS (J2000)DEC,Mag U,Mag B,Mag V,Mag R,Mag I,Sp type,#ref 1850 - 2019,#notes
0,1,TYC 8944-2490-1,136.38,*,09 25 19.1326773679,-62 29 34.096719650,NaN,13.38,12.03,NaN,NaN,~,0,0
1,2,TYC 8944-2592-1,218.55,*,09 25 19.9386281509,-62 28 08.034529539,NaN,12.90,12.16,NaN,NaN,~,0,0


In [26]:
df = interesting.copy()
tmp = df.apply(lambda source: get_response(source.ra, source.dec)[0], axis=1)
df.insert(2, 'candidate', tmp)

In [27]:
df.head()

,index,source_id,candidate,ra,dec,mean_raw_peak_flux,std_raw_peak_flux,Mod,length,sig,grad,fit_error,y_int,avg_sig,avg_grad,avg_fit_error,avg_y_int,avg_std,avg_median,avg_Mod
0,2490,9626,TYC 8995-1855-1,203.152823,-62.778506,0.765738,0.303694,39.660271,94,3.983624,0.000438,0.000110,-1074.363164,1.504227,0.000356,0.000237,-874.196992,0.252276,0.697101,36.189312
1,2676,10057,TYC 8944-2490-1,141.356216,-62.528659,0.344761,0.101120,29.330620,36,8.899923,0.000300,0.000034,-737.705074,5.960626,0.000328,0.000055,-805.570112,0.110607,0.339858,32.545129
2,3289,11643,2MASS J11121911-6120121,168.081167,-61.342199,0.848598,0.337037,39.716871,75,6.032429,0.000696,0.000115,-1710.497678,2.617023,0.000529,0.000202,-1300.078987,0.241509,0.759459,31.800188
3,3294,11656,IRAS 13234-6104,201.677521,-61.331384,0.827691,0.411734,49.744936,38,-0.079716,-0.000022,0.000275,54.666834,0.451813,0.000200,0.000443,-491.436920,0.314633,0.792390,39.706788
4,3482,12131,[SWK2009] K154,243.560256,-61.021059,2.573485,1.978635,76.885424,70,1.129493,0.000947,0.000838,-2323.165422,-0.670967,-0.001408,0.002098,3461.891655,1.738782,2.528826,68.758477


In [36]:
urls = df.apply(lambda source: query(source.ra, source.dec), axis=1)
df['simbad_url'] = urls

In [38]:
df.head()

,index,source_id,candidate,ra,dec,mean_raw_peak_flux,std_raw_peak_flux,Mod,length,sig,...,fit_error,y_int,avg_sig,avg_grad,avg_fit_error,avg_y_int,avg_std,avg_median,avg_Mod,simbad_url
0,2490,9626,TYC 8995-1855-1,203.152823,-62.778506,0.765738,0.303694,39.660271,94,3.983624,...,0.000110,-1074.363164,1.504227,0.000356,0.000237,-874.196992,0.252276,0.697101,36.189312,http://simbad.u-strasbg.fr/simbad/sim-coo?Coor...
1,2676,10057,TYC 8944-2490-1,141.356216,-62.528659,0.344761,0.101120,29.330620,36,8.899923,...,0.000034,-737.705074,5.960626,0.000328,0.000055,-805.570112,0.110607,0.339858,32.545129,http://simbad.u-strasbg.fr/simbad/sim-coo?Coor...
2,3289,11643,2MASS J11121911-6120121,168.081167,-61.342199,0.848598,0.337037,39.716871,75,6.032429,...,0.000115,-1710.497678,2.617023,0.000529,0.000202,-1300.078987,0.241509,0.759459,31.800188,http://simbad.u-strasbg.fr/simbad/sim-coo?Coor...
3,3294,11656,IRAS 13234-6104,201.677521,-61.331384,0.827691,0.411734,49.744936,38,-0.079716,...,0.000275,54.666834,0.451813,0.000200,0.000443,-491.436920,0.314633,0.792390,39.706788,http://simbad.u-strasbg.fr/simbad/sim-coo?Coor...
4,3482,12131,[SWK2009] K154,243.560256,-61.021059,2.573485,1.978635,76.885424,70,1.129493,...,0.000838,-2323.165422,-0.670967,-0.001408,0.002098,3461.891655,1.738782,2.528826,68.758477,http://simbad.u-strasbg.fr/simbad/sim-coo?Coor...


In [40]:
df.insert(3, 'isPulsar', df.candidate.str.startswith('PSR'))

In [41]:
df.to_excel('~/Downloads/sources.xlsx')

In [25]:
interesting.iloc[:10, :].apply(lambda source: get_response(source.ra, source.dec)[0], axis=1)

0    (TYC 8995-1855-1, http://simbad.u-strasbg.fr/s...
1    (TYC 8944-2490-1, http://simbad.u-strasbg.fr/s...
2    (2MASS J11121911-6120121, http://simbad.u-stra...
3    (IRAS 13234-6104, http://simbad.u-strasbg.fr/s...
4    ([SWK2009] K154, http://simbad.u-strasbg.fr/si...
5    (TYC 8927-3589-1, http://simbad.u-strasbg.fr/s...
6    (DAY-I C1109-604-07773, http://simbad.u-strasb...
7    (NGC 6156, http://simbad.u-strasbg.fr/simbad/s...
8    ([NWN2004] 16, http://simbad.u-strasbg.fr/simb...
9    (TYC 9036-1046-1, http://simbad.u-strasbg.fr/s...
dtype: object

In [18]:
interesting.shape

(105, 19)